In [12]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
import os
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
from itertools import product, combinations
import math
import warnings
# Import some libraries
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import georasters as gr

In [13]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Philadelphia']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)
park_entry_point_merge = 0 # (default)

grid_block_size = 100 # in meters2 rounded to 100m
nn_iter = 10
#block_combinations = 100
block_network = 250000

In [14]:
# Block 1 Extract city boundaries, download grids in cities' country

warnings.filterwarnings("ignore")
bound_df = ox.geocoder.geocode_to_gdf(cities)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')

start_time = time.time()

# Get unique ISO countries, so all country-grids are only loaded once (i.e. Philadelphia, Denver and Washington DC in the USA)
C = pd.DataFrame(Countries)
start_time = time.time()
iso_countries = []
print('required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)')
for i in range(len(cities)): 
    countries = bound_df['display_name'][i].rsplit(',')[-1][1:]
    iso = C[C['name'] == countries].iloc[0,1]
    if iso not in iso_countries:
        iso_countries.append(iso)
        
        # Search for data
        products = Product(iso)
        Results = products.description_contains('people per grid-cell 2020')
        list1 = []
        for p in Results:
            prints = '%s/%s\t%s\t%s' % (p.idx, p.country_name,p.dataset_name,p.path)
            list1.append(prints)
        print('wpgpDownload download -i',iso,'--id',list1[0].split("\t")[0].split('/')[0])

print('')
blocks = []
for i in iso_countries:
    # Get the country by saved path ()
    # You can also use sys.path: path = sys.path[0] +'\\' + str('ISL').lower() + '_ppp_2020.tif'
    # The sys.path is the path Python saves the python file automatically before you
    path = 'D:\\Dumps\\WorldPoP_Grids\\' + i.lower() + '_ppp_2020.tif'
    block = gr.from_file(path)
    blocks.append(block)
    print(i,'extracted', round((time.time() - start_time)/60,2),'mns')
bound_df

required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)
wpgpDownload download -i USA --id 4983

USA extracted 4.22 mns


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...",40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,40.00241,-75.139364,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,0.823797


In [15]:
# Block 2 population grids extraction
start_time = time.time()
clips = []
grids = []
print(str(grid_block_size) + 'm resolution grids extraction')
for i in range(len(cities)):
    iso = C[bound_df['display_name'][i].rsplit(',')[-1][1:] == C['name']].iloc[0,1]
    contains = [j for j, x in enumerate(iso_countries) if x == iso][0]
    
    # Clip the city from the country
    clipped = blocks[contains].clip(bound_df['geometry'][i])
    clipped = clipped[0].to_geopandas()
    
    # Get dissolvement_key for dissolvement. 
    clipped['row3'] = np.floor(clipped['row']/(grid_block_size/100)).astype(int)
    clipped['col3'] = np.floor(clipped['col']/(grid_block_size/100)).astype(int)
    clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)
    clips.append(clipped)
        
    # Dissolve into block by block grids
    popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

    # Get those grids populations and area. Only blocks with population and full blocks
    popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
    popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
    popgrid = popgrid[popgrid['population'] > 0]
    popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]
    
    # Get centroids and coords
    popgrid['centroid'] = popgrid['geometry'].centroid
    popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    popgrid = popgrid.reset_index()
    
    minx = popgrid.bounds['minx']
    maxx = popgrid.bounds['maxx']
    miny = popgrid.bounds['miny']
    maxy = popgrid.bounds['maxy']

    # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
    # I therefore create one polygon
    Poly = []
    for k in range(len(popgrid)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    popgrid['geometry'] = Poly
    
    grids.append(popgrid)
    
    print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    
grids[0]

100m resolution grids extraction
Philadelphia 1.04 mns


,dissolve_key,geometry,row3,col3,population,area_m,centroid,centroid_m,grid_lon,grid_lat
0,0-318,"POLYGON ((-75.01542 40.13875, -75.01458 40.138...",0,318,10,14838,POINT (-75.01500 40.13833),POINT (-5675320.992 8459941.626),-5.675321e+06,8.459942e+06
1,1-317,"POLYGON ((-75.01625 40.13792, -75.01542 40.137...",1,317,4,14839,POINT (-75.01583 40.13750),POINT (-5675486.353 8459999.741),-5.675486e+06,8.460000e+06
2,1-318,"POLYGON ((-75.01542 40.13792, -75.01458 40.137...",1,318,12,14839,POINT (-75.01500 40.13750),POINT (-5675453.122 8459898.329),-5.675453e+06,8.459898e+06
3,1-319,"POLYGON ((-75.01458 40.13792, -75.01375 40.137...",1,319,9,14838,POINT (-75.01417 40.13750),POINT (-5675419.890 8459796.917),-5.675420e+06,8.459797e+06
4,1-320,"POLYGON ((-75.01375 40.13792, -75.01292 40.137...",1,320,6,14838,POINT (-75.01333 40.13750),POINT (-5675386.655 8459695.507),-5.675387e+06,8.459696e+06
...,...,...,...,...,...,...,...,...,...,...
34885,99-95,"POLYGON ((-75.20125 40.05625, -75.20042 40.056...",99,95,2,14926,POINT (-75.20083 40.05583),POINT (-5695801.873 8478348.566),-5.695802e+06,8.478349e+06
34886,99-96,"POLYGON ((-75.20042 40.05625, -75.19958 40.056...",99,96,4,14926,POINT (-75.20000 40.05583),POINT (-5695768.954 8478246.652),-5.695769e+06,8.478247e+06
34887,99-97,"POLYGON ((-75.19958 40.05625, -75.19875 40.056...",99,97,8,14926,POINT (-75.19917 40.05583),POINT (-5695736.034 8478144.739),-5.695736e+06,8.478145e+06
34888,99-98,"POLYGON ((-75.19875 40.05625, -75.19792 40.056...",99,98,8,14926,POINT (-75.19833 40.05583),POINT (-5695703.111 8478042.827),-5.695703e+06,8.478043e+06


In [16]:
# Block 3 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
    #graphs.append(graph)
    
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    #road_nodes.append(road_node)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    #road_edges.append(road_edge)
    
    # Exclude highways and ramps on edges    
    edge_filter = road_edge[(road_edge['highway'].str.contains('motorway') | 
          (road_edge['highway'].str.contains('trunk') & 
           road_edge['maxspeed'].astype(str).str.contains(
               '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
    road_edges.append(edge_filter)
    
    # Exclude isolated nodes
    fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
    newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
    road_nodes.append(newnodes)
    
    # Get only necessary road connections columns for network performance
    road_con = edge_filter[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    
    # formatting to graph again.
    newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
    edge_filter = edge_filter.set_index(['from','to','keys'])
    edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]
    graph2 = ox.graph_from_gdfs(newnodes, edge_filter)
                            
    graphs.append(graph2)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    
road_edges[0]

Philadelphia done 10.98 mns


,from,to,keys,osmid,name,highway,oneway,length,geometry,ref,...,width,bridge,service,tunnel,maxspeed,access,junction,area,geometry_m,key
0,103237949,103353127,0,11591597,New Jersey Avenue,residential,False,111.799,"LINESTRING (-74.96444 40.03625, -74.96557 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689617...",103237949-103353127
1,103237949,103353090,0,11591597,New Jersey Avenue,residential,False,107.489,"LINESTRING (-74.96444 40.03625, -74.96337 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689364...",103237949-103353090
2,103237949,103237976,0,11580386,Cleveland Avenue,residential,False,168.717,"LINESTRING (-74.96444 40.03625, -74.96404 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689552...",103237949-103237976
3,103237976,103238007,0,11580386,Cleveland Avenue,residential,False,151.349,"LINESTRING (-74.96340 40.03496, -74.96245 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689797...",103237976-103238007
4,103237976,103590312,0,11610261,2nd Street,residential,False,108.508,"LINESTRING (-74.96340 40.03496, -74.96448 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689779...",103237976-103590312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338097,9726316545,5550941352,0,579660432,NaN,service,False,39.965,"LINESTRING (-75.24128 40.06507, -75.24111 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695935...",9726316545-5550941352
338098,9726316545,9726316548,0,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24128 40.06507, -75.24148 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695962...",9726316545-9726316548
338099,9726316548,9726316545,0,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24175 40.06470, -75.24175 40.0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5696003.982 8483808.258, -5695998...",9726316548-9726316545
338100,9728655691,110122187,0,12149073,South 10th Street,residential,True,3.250,"LINESTRING (-75.15829 39.94437, -75.15830 39.9...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5711864.888 8467388.116, -5711869...",9728655691-110122187


In [17]:
# Block 4 city greenspace

parks_in_range = list()
for i in range(len(cities)):
    path = 'D:/Dumps/Greenspace/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_Greenspace.gpkg'
    greenspace = gpd.read_file(path).to_crs(4326)
    
    warnings.filterwarnings("ignore")

    green_buffer = gpd.GeoDataFrame(geometry = greenspace['geometry'].to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')

    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
    
    # Get the park buffer
    
    buffer = gpd.GeoDataFrame(geometry = bound_df.to_crs(3043).buffer(np.max(thresholds)).to_crs(4326), crs = 3043)
    parks_t = parks[parks.intersection(bound_df['geometry'].to_crs(3043).buffer(1000).to_crs(4326)[0]).area > 0]
    parks_t = parks_t.reset_index()
    
    parks_in_range.append(parks_t)
    
parks_in_range[0]

,components,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,...,IUCNCtSrc,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff
0,5,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,...,GAP - Default,2020,1983,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061..."
1,6,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,GAP - Default,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916..."
2,7,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,GAP - Default,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921..."
3,8,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,Fee,Fee,FED,NPS,NPS,FED,NPS,...,GAP - Default,2020,1978,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961..."
4,9,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,Fee,Fee,FED,NPS,NPS,FED,NPS,...,GAP - Default,2020,None,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,734,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,GAP - Default,2020,None,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949..."
469,735,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,GAP - Default,2020,None,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968..."
470,736,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...",242023,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,GAP - Default,2020,None,None,None,None,458.291779,1.188739e+04,"POLYGON ((-75.25664 39.94598, -75.25669 39.945...","POLYGON ((-75.25669 39.94597, -75.25671 39.945..."
471,739,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...",242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,...,GAP - Default,2020,None,None,None,None,552.520121,1.592591e+04,"POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27789 39.96685, -75.27788 39.966..."


In [18]:
# Block 5 park entry points
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 100 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry']], left_on = 'Park_No', right_index = True)
    #ParkRoad = pd.merge(ParkRoad, parks_in_range[j][thresholds_str], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_sqr2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_sqr3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_sqr5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Philadelphia 0.0 % done 0.02  mns
Philadelphia 21.1 % done 1.78  mns
Philadelphia 42.3 % done 2.56  mns
Philadelphia 63.4 % done 3.22  mns
Philadelphia 84.6 % done 3.99  mns
Philadelphia 100 % done 5.34  mns


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5
0,109991185,40.053816,-75.240687,3,NaN,NaN,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),109991185,0,...,8.483120e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.23914 40.05656, -75.23880 40.056...",198723.068185,4.192678e+06,0.047398,1.990313,1.410785,1.257884,1.147583
1,109991191,40.054803,-75.239081,3,NaN,NaN,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),109991191,0,...,8.482975e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.24278 40.05572, -75.24265 40.055...",291442.444263,4.192678e+06,0.069512,2.918945,1.708492,1.429142,1.238925
2,109991199,40.055776,-75.237501,3,NaN,NaN,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),109991199,0,...,8.482831e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.24077 40.05416, -75.24096 40.054...",245064.061026,4.192678e+06,0.058450,2.454442,1.566666,1.348914,1.196714
3,109991204,40.056232,-75.236783,3,NaN,NaN,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),109991204,0,...,8.482767e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.24005 40.05461, -75.24024 40.054...",187511.352028,4.192678e+06,0.044724,1.878022,1.370409,1.233768,1.134332
4,110169862,40.063972,-75.249174,3,NaN,NaN,POINT (-75.24917 40.06397),POINT (-5696411.964 8484679.155),110169862,0,...,8.484679e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.24676 40.06163, -75.24707 40.061...",194283.946127,4.192678e+06,0.046339,1.945853,1.394938,1.248447,1.142410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7289,3425873720,39.966878,-75.277524,3,NaN,NaN,POINT (-75.27752 39.96688),POINT (-5713004.185 8483178.234),3425873720,471,...,8.483178e+06,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...","POLYGON ((-75.27647 39.96684, -75.27650 39.966...",19734.719624,1.973472e+04,1.000000,0.197653,0.444582,0.582507,0.723071
7290,110006563,39.961942,-75.273003,3,NaN,NaN,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),110006563,472,...,8.482370e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755
7291,110133414,39.961600,-75.274141,3,NaN,NaN,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),110133414,472,...,8.482492e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755
7292,923789083,39.962719,-75.274196,3,NaN,NaN,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),923789083,472,...,8.482556e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755


In [19]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_area,share_walked,size_infl_factor,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5,geometry_m_buffer,components,comp_centroid,centr_dist
0,109991185,40.053816,-75.240687,3,NaN,NaN,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),109991185,0,...,4.192678e+06,0.047398,1.990313,1.410785,1.257884,1.147583,POLYGON EMPTY,0,POINT (-75.24069 40.05382),0.0
1,109991191,40.054803,-75.239081,3,NaN,NaN,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),109991191,0,...,4.192678e+06,0.069512,2.918945,1.708492,1.429142,1.238925,POLYGON EMPTY,1,POINT (-75.23908 40.05480),0.0
2,109991199,40.055776,-75.237501,3,NaN,NaN,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),109991199,0,...,4.192678e+06,0.058450,2.454442,1.566666,1.348914,1.196714,POLYGON EMPTY,2,POINT (-75.23750 40.05578),0.0
3,109991204,40.056232,-75.236783,3,NaN,NaN,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),109991204,0,...,4.192678e+06,0.044724,1.878022,1.370409,1.233768,1.134332,POLYGON EMPTY,3,POINT (-75.23678 40.05623),0.0
4,110169862,40.063972,-75.249174,3,NaN,NaN,POINT (-75.24917 40.06397),POINT (-5696411.964 8484679.155),110169862,0,...,4.192678e+06,0.046339,1.945853,1.394938,1.248447,1.142410,POLYGON EMPTY,4,POINT (-75.24917 40.06397),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7289,3425873720,39.966878,-75.277524,3,NaN,NaN,POINT (-75.27752 39.96688),POINT (-5713004.185 8483178.234),3425873720,471,...,1.973472e+04,1.000000,0.197653,0.444582,0.582507,0.723071,POLYGON EMPTY,7289,POINT (-75.27752 39.96688),0.0
7290,110006563,39.961942,-75.273003,3,NaN,NaN,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),110006563,472,...,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755,POLYGON EMPTY,7290,POINT (-75.27300 39.96194),0.0
7291,110133414,39.961600,-75.274141,3,NaN,NaN,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),110133414,472,...,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755,POLYGON EMPTY,7291,POINT (-75.27414 39.96160),0.0
7292,923789083,39.962719,-75.274196,3,NaN,NaN,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),923789083,472,...,4.717632e+04,1.000000,0.472495,0.687383,0.778871,0.860755,POLYGON EMPTY,7292,POINT (-75.27420 39.96272),0.0


In [20]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    #blockA = block_combinations
    print(cities[l])
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    
    # Reduce the size of combinations per iteration
    len4 = 1
    len5 = len1 * len2
    blockC = len5
    while blockC > 10000000:
        blockC = len5 / len4
        #print(blockC, len4)
        len4 = len4+1
        
    # Amount of grids taken per iteration block
    block = round(len1 / len4)
    
    output = pd.DataFrame()
    len_mat = 0
    len_df = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len4):
        # Check all grid-park combinations per block
        l1, l2 = range(i*block,(i+1)*block), range(0,len2)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_sqr2','size_infl_sqr3','size_infl_sqr5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_sqr2'])
        infl3 = list(node_merged['size_infl_sqr3'])
        infl5 = list(node_merged['size_infl_sqr5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])
        len_mat = len_mat + len(mat_df)
        len_df = len_df + len(node_merged)
            
        if ( (i+1) % 1 == 0) | ((i+1) == len4): 
            print((i+1),'/',len4,'comb. done',round((time.time() - start_time) / 60,2),' mns')
            
            print('of',len_df,'within a Gravity model variant in one of',thresholds,'m threshold:',len_mat)
            len_mat = 0
            len_df = 0
    # Renaming columns
    print('total combinations within distance',len(output))
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5',
                      'parkshare_walked','park_area','walk_area_m2']
    
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        try:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        except:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    #output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
    #                                               ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]  

Philadelphia
1 / 27 comb. done 1.5  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 59050
2 / 27 comb. done 2.54  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 54750
3 / 27 comb. done 3.55  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 72357
4 / 27 comb. done 4.56  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 95515
5 / 27 comb. done 5.57  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 96850
6 / 27 comb. done 6.6  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 84480
7 / 27 comb. done 7.6  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 87824
8 / 27 comb. done 8.6  mns
of 9423848 within a Gravity model variant in one of [300, 600, 1000] m threshold: 68256
9 / 27 comb. done 9.59  mns
of 9423848 within a Gravity model v

,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_coords_centroid,Grid_m_centroid,Park_No,...,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5,grid_osm,geometry,geometry_m
0,246,588.711176,417.293464,468.017201,513.000751,246,0,POINT (-75.24500 40.05500),POINT (-5697676.322 8483708.862),0,...,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.410785,1.257884,1.147583,5550941817,POINT (-75.24315 40.05491),POINT (-5697618.342 8483477.646)
1,247,489.160666,346.729529,388.875930,426.252804,247,0,POINT (-75.24417 40.05500),POINT (-5697643.511 8483606.888),0,...,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.410785,1.257884,1.147583,5550941817,POINT (-75.24315 40.05491),POINT (-5697618.342 8483477.646)
2,248,393.608019,278.999381,312.912904,342.988579,248,0,POINT (-75.24333 40.05500),POINT (-5697610.697 8483504.915),0,...,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.410785,1.257884,1.147583,5550941817,POINT (-75.24315 40.05491),POINT (-5697618.342 8483477.646)
3,504,557.494365,395.166194,443.200271,485.798537,504,0,POINT (-75.24500 40.05417),POINT (-5697809.025 8483666.164),0,...,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,1.410785,1.257884,1.147583,5550941817,POINT (-75.24315 40.05491),POINT (-5697618.342 8483477.646)
4,1538,761.539231,445.737656,532.864732,614.677192,246,1,POINT (-75.24500 40.05500),POINT (-5697676.322 8483708.862),0,...,POINT (-75.23908 40.05480),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697474.612 8482974.522),291442.444263,1.708492,1.429142,1.238925,5550941817,POINT (-75.24315 40.05491),POINT (-5697618.342 8483477.646)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593425,8594218,928.279618,1211.976137,1108.891321,1032.773097,34718,6651,POINT (-75.14250 40.05583),POINT (-5693492.657 8471217.220),408,...,POINT (-75.14622 40.05107),"POLYGON ((-75.14640 40.04963, -75.14643 40.049...",POINT (-5694397.065 8471426.383),58572.853245,0.765922,0.837124,0.898822,5546506201,POINT (-75.14233 40.05579),POINT (-5693492.537 8471194.283)
1593426,8595510,970.421527,1266.997261,1159.232615,1079.658787,34718,6652,POINT (-75.14250 40.05583),POINT (-5693492.657 8471217.220),408,...,POINT (-75.14700 40.05118),"POLYGON ((-75.14640 40.04963, -75.14643 40.049...",POINT (-5694412.221 8471527.252),58572.853245,0.765922,0.837124,0.898822,5546506201,POINT (-75.14233 40.05579),POINT (-5693492.537 8471194.283)
1593427,8600678,956.939067,1249.394353,1143.126925,1064.658649,34718,6656,POINT (-75.14250 40.05583),POINT (-5693492.657 8471217.220),408,...,POINT (-75.14677 40.05115),"POLYGON ((-75.14640 40.04963, -75.14643 40.049...",POINT (-5694407.706 8471497.251),58572.853245,0.765922,0.837124,0.898822,5546506201,POINT (-75.14233 40.05579),POINT (-5693492.537 8471194.283)
1593428,8601970,906.887108,1184.045747,1083.336554,1008.972501,34718,6657,POINT (-75.14250 40.05583),POINT (-5693492.657 8471217.220),408,...,POINT (-75.14574 40.05101),"POLYGON ((-75.14640 40.04963, -75.14643 40.049...",POINT (-5694387.566 8471364.129),58572.853245,0.765922,0.837124,0.898822,5546506201,POINT (-75.14233 40.05579),POINT (-5693492.537 8471194.283)


In [21]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Philadelphia 1593430


In [22]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = block_network

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    1217624
    # Divide in chunks of block for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        s_mat5 = list([])
        mat_nn = []
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(-1)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
                s_mat5.append(1)
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                    s_mat5.append(0)
                except:
                    # Otherwise the nearest node is taken, which is iterated X times at max, check assumptions, block #0 
                    # Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < nn_iter and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                            s_mat4.append(str(len3)+'grid > n-park')
                            s_mat5.append(1)
                        except:
                            try:
                                alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                                s_mat4.append(str(len3)+'n-park > grid')
                                s_mat5.append(0)
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                    s_mat4.append(str(len3)+'n-grid > park')
                                    s_mat5.append(1)
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                        s_mat4.append(str(len3)+'park > n-grid')
                                        s_mat5.append(0)
                                    except:
                                        if len3 == nn_iter:
                                            #print(i+block*k,i+block*k,
                                            #      'No route between grid and park-entry and their both 10 alternatives')
                                            pass
                                        pass
                    #print(len(alt_route))
                    if len(alt_route) == 0: 
                        alt = alt_route 
                    else: 
                        alt = alt_route[0]
                    len4 = len(alt)
                    #print(len4)
                    #mat_nn = []
                    if len4 > 0:
                        #print('for index',i+block*k,'nearest node found between', 
                        #                       alt[0],'and',alt[-1])
                        mat_nn.append(i+block*k)
                        s_mat.append(alt)
                        shortest_to = list(alt[1:len(alt)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4)))
                        s_mat3.append(list(np.arange(0, len4)))
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')
                        s_mat5.append(2)
                        print(i+block*k,'No route')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')
        print('for', len(mat_nn),' routes nearest nodes found')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # dissolve route
        routes2 = routes[['route','geometry']].dissolve('route')
        
        # get used grid- and parkosm. Differs at NN-route.
        route_reset = routes.reset_index()
        origin = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmin()),]
        origin = origin.reset_index().iloc[:,-1]
        dest = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmax()),]
        dest = dest.reset_index().iloc[:,-1]
        
        # grid > park = 1, park > grid = 0, no way = 2, detailed way in way_calc.
        routes2['G1-P0'] = s_mat5
        routes2['real_grid'] = np.where(routes2['G1-P0'] == 1, origin, dest)
        routes2['real_park'] = np.where(routes2['G1-P0'] == 1, dest, origin)
        routes2['way_calc.'] = s_mat4
        
        # get route cost, steps, additional information.
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                              'Grid_m_centroid','walk_area_m2','size_infl_sqr2','size_infl_sqr3',
                                              'size_infl_sqr5','raw euclidean']],
                                                left_index = True, right_index = True)
        routes2 = pd.merge(routes2, road_nodes[j]['geometry_m'], how = 'left', left_on = 'real_grid', right_index = True)
        
        # calculate distance of used road-entry for grid-centroid.
        routes2['real_G-entry'] = round(gpd.GeoSeries(routes2['Grid_m_centroid'], crs = 3043
                                                       ).distance(routes2['geometry_m']),3)
        
        # Calculcate total route cost for the four gravity variants
        routes2['raw_Tcost'] = routes2['route_cost'] + routes2['real_G-entry']
        routes2['grav2_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr2']
        routes2['grav3_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr3']
        routes2['grav5_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]

Philadelphia 1 / 7 range 0 - 250000
0.0 % done 0.05 mns
0.63 % done 0.28 mns
1.26 % done 0.44 mns
1.88 % done 0.77 mns
2.51 % done 0.94 mns
3.14 % done 1.08 mns
3.77 % done 1.18 mns
4.39 % done 1.32 mns
5.02 % done 1.54 mns
5.65 % done 1.7 mns
6.28 % done 1.91 mns
6.9 % done 2.14 mns
7.53 % done 2.28 mns
8.16 % done 2.45 mns
8.79 % done 2.97 mns
9.41 % done 3.2 mns
10.04 % done 3.44 mns
10.67 % done 3.66 mns
11.3 % done 3.98 mns
11.92 % done 4.11 mns
12.55 % done 4.4 mns
13.18 % done 4.63 mns
13.81 % done 4.86 mns
14.43 % done 5.16 mns
15.06 % done 5.42 mns
for 165  routes nearest nodes found
15.69 % pathfinding done 5.64 mns
formatting done 7.26 mns
dissolving done 8.69 mns
Philadelphia 2 / 7 range 250000 - 500000
15.69 % done 8.69 mns
16.32 % done 8.94 mns
16.94 % done 9.14 mns
17.57 % done 9.26 mns
18.2 % done 9.39 mns
18.83 % done 9.6 mns
19.45 % done 9.84 mns
20.08 % done 10.11 mns
20.71 % done 10.37 mns
21.34 % done 10.62 mns
21.97 % done 10.88 mns
22.59 % done 11.06 mns
23.22 % 

1299546 No route between grid and park-entry and their both 10 alternatives
1299549 No route between grid and park-entry and their both 10 alternatives
1299550 No route between grid and park-entry and their both 10 alternatives
1299551 No route between grid and park-entry and their both 10 alternatives
1299552 No route between grid and park-entry and their both 10 alternatives
1299553 No route between grid and park-entry and their both 10 alternatives
1299555 No route between grid and park-entry and their both 10 alternatives
1299556 No route between grid and park-entry and their both 10 alternatives
1299557 No route between grid and park-entry and their both 10 alternatives
1299558 No route between grid and park-entry and their both 10 alternatives
1299559 No route between grid and park-entry and their both 10 alternatives
1299560 No route between grid and park-entry and their both 10 alternatives
1299561 No route between grid and park-entry and their both 10 alternatives
1299562 No r

1299680 No route between grid and park-entry and their both 10 alternatives
1299681 No route between grid and park-entry and their both 10 alternatives
1299682 No route between grid and park-entry and their both 10 alternatives
1299683 No route between grid and park-entry and their both 10 alternatives
1299684 No route between grid and park-entry and their both 10 alternatives
1299685 No route between grid and park-entry and their both 10 alternatives
1299687 No route between grid and park-entry and their both 10 alternatives
1299688 No route between grid and park-entry and their both 10 alternatives
1299689 No route between grid and park-entry and their both 10 alternatives
1299690 No route between grid and park-entry and their both 10 alternatives
1299691 No route between grid and park-entry and their both 10 alternatives
1299692 No route between grid and park-entry and their both 10 alternatives
1299693 No route between grid and park-entry and their both 10 alternatives
1299694 No r

1299829 No route between grid and park-entry and their both 10 alternatives
1299830 No route between grid and park-entry and their both 10 alternatives
1299831 No route between grid and park-entry and their both 10 alternatives
1299832 No route between grid and park-entry and their both 10 alternatives
1299833 No route between grid and park-entry and their both 10 alternatives
1299834 No route between grid and park-entry and their both 10 alternatives
1299835 No route between grid and park-entry and their both 10 alternatives
1299837 No route between grid and park-entry and their both 10 alternatives
1299838 No route between grid and park-entry and their both 10 alternatives
1299839 No route between grid and park-entry and their both 10 alternatives
1299840 No route between grid and park-entry and their both 10 alternatives
1299844 No route between grid and park-entry and their both 10 alternatives
1299845 No route between grid and park-entry and their both 10 alternatives
1299847 No r

1299964 No route between grid and park-entry and their both 10 alternatives
1299965 No route between grid and park-entry and their both 10 alternatives
1299966 No route between grid and park-entry and their both 10 alternatives
1299967 No route between grid and park-entry and their both 10 alternatives
1299971 No route between grid and park-entry and their both 10 alternatives
1299972 No route between grid and park-entry and their both 10 alternatives
1299973 No route between grid and park-entry and their both 10 alternatives
1299974 No route between grid and park-entry and their both 10 alternatives
1299975 No route between grid and park-entry and their both 10 alternatives
1299976 No route between grid and park-entry and their both 10 alternatives
1299977 No route between grid and park-entry and their both 10 alternatives
1299978 No route between grid and park-entry and their both 10 alternatives
1299979 No route between grid and park-entry and their both 10 alternatives
1299980 No r

...

1321759 No route between grid and park-entry and their both 10 alternatives
1321760 No route between grid and park-entry and their both 10 alternatives
1321761 No route between grid and park-entry and their both 10 alternatives
1321762 No route between grid and park-entry and their both 10 alternatives
1321763 No route between grid and park-entry and their both 10 alternatives
1321764 No route between grid and park-entry and their both 10 alternatives
1321765 No route between grid and park-entry and their both 10 alternatives
1321768 No route between grid and park-entry and their both 10 alternatives
1321769 No route between grid and park-entry and their both 10 alternatives
1321770 No route between grid and park-entry and their both 10 alternatives
1321771 No route between grid and park-entry and their both 10 alternatives
1321772 No route between grid and park-entry and their both 10 alternatives
1321773 No route between grid and park-entry and their both 10 alternatives
1321774 No r

1321886 No route between grid and park-entry and their both 10 alternatives
1321887 No route between grid and park-entry and their both 10 alternatives
1321888 No route between grid and park-entry and their both 10 alternatives
1321889 No route between grid and park-entry and their both 10 alternatives
1321890 No route between grid and park-entry and their both 10 alternatives
1321891 No route between grid and park-entry and their both 10 alternatives
1321892 No route between grid and park-entry and their both 10 alternatives
1321893 No route between grid and park-entry and their both 10 alternatives
1321894 No route between grid and park-entry and their both 10 alternatives
1321896 No route between grid and park-entry and their both 10 alternatives
1321897 No route between grid and park-entry and their both 10 alternatives
1321898 No route between grid and park-entry and their both 10 alternatives
1321899 No route between grid and park-entry and their both 10 alternatives
1321903 No r

1322185 No route between grid and park-entry and their both 10 alternatives
1322186 No route between grid and park-entry and their both 10 alternatives
1322188 No route between grid and park-entry and their both 10 alternatives
1322189 No route between grid and park-entry and their both 10 alternatives
1322190 No route between grid and park-entry and their both 10 alternatives
1322191 No route between grid and park-entry and their both 10 alternatives
1322195 No route between grid and park-entry and their both 10 alternatives
1322196 No route between grid and park-entry and their both 10 alternatives
1322198 No route between grid and park-entry and their both 10 alternatives
1322199 No route between grid and park-entry and their both 10 alternatives
1322200 No route between grid and park-entry and their both 10 alternatives
1322201 No route between grid and park-entry and their both 10 alternatives
1322202 No route between grid and park-entry and their both 10 alternatives
1322203 No r

99.78 % done 98.79 mns
for 0  routes nearest nodes found
100.0 % pathfinding done 98.9 mns
formatting done 99.49 mns
dissolving done 100.02 mns


,geometry,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,grid_osm,Park_No,...,size_infl_sqr3,size_infl_sqr5,raw euclidean,geometry_m,real_G-entry,raw_Tcost,grav2_Tcost,grav3_Tcost,grav5_Tcost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.24315 40.05491, -75.2427...",1,5550941817,109991185,normal way,303.466,3,246,5550941817,0,...,1.257884,1.147583,588.711176,POINT (-5697618.342 8483477.646),238.375,541.841,384.070691,430.756063,472.158253,246-0
1,"MULTILINESTRING ((-75.24315 40.05491, -75.2427...",1,5550941817,109991185,normal way,303.466,3,247,5550941817,0,...,1.257884,1.147583,489.160666,POINT (-5697618.342 8483477.646),131.670,435.136,308.435471,345.927071,379.175909,247-0
2,"MULTILINESTRING ((-75.24315 40.05491, -75.2427...",1,5550941817,109991185,normal way,303.466,3,248,5550941817,0,...,1.257884,1.147583,393.608019,POINT (-5697618.342 8483477.646),28.321,331.787,235.179070,263.766053,289.117970,248-0
3,"MULTILINESTRING ((-75.24315 40.05491, -75.2427...",1,5550941817,109991185,normal way,303.466,3,504,5550941817,0,...,1.257884,1.147583,557.494365,POINT (-5697618.342 8483477.646),268.140,571.606,405.168880,454.418824,498.095364,504-0
4,"MULTILINESTRING ((-75.24315 40.05491, -75.2427...",1,5550941817,109991191,normal way,478.764,4,246,5550941817,0,...,1.429142,1.238925,761.539231,POINT (-5697618.342 8483477.646),238.375,717.139,419.749691,501.796973,578.839499,246-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593425,"MULTILINESTRING ((-75.14233 40.05579, -75.1423...",1,5546506201,109816586,normal way,814.805,15,34718,5546506201,408,...,0.837124,0.898822,928.279618,POINT (-5693492.537 8471194.283),22.937,837.742,1093.768831,1000.738155,932.043948,34718-408
1593426,"MULTILINESTRING ((-75.14233 40.05579, -75.1423...",1,5546506201,109816593,normal way,881.866,14,34718,5546506201,408,...,0.837124,0.898822,970.421527,POINT (-5693492.537 8471194.283),22.937,904.803,1181.324703,1080.846950,1006.653791,34718-408
1593427,"MULTILINESTRING ((-75.14233 40.05579, -75.1423...",1,5546506201,7205122808,normal way,901.963,15,34718,5546506201,408,...,0.837124,0.898822,956.939067,POINT (-5693492.537 8471194.283),22.937,924.900,1207.563655,1104.854144,1029.013046,34718-408


In [23]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        Rclean = Routes[n][Routes[n]['way_calc.'] != 'no way'].reset_index()
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Rclean.iloc[Rclean.groupby('gridpark_no')[(str(l1[i]) +'_Tcost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_Tcost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['population','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_Tcost'] <= t]
            thold[score] = t - thold[l1[i] + '_Tcost']
            thold['pop' + score] = thold[score] * thold['population']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['population'] / thold['walk_area_ha' + str2]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + str2]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2].sum()
            
            gs['walkha_person_' + score] = locals()[str1].groupby('Grid_No')['walkha_person' + str2].mean()

            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['population'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_lines/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_lines/')
                
            gs.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_lines/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['population'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['population'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/'):
            os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/')
            
        gridsc.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_csv/'):
            os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_csv/')
        
        gridsc.to_csv('D:/Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/radius-euclidean adjusted popgrid access.csv')

popg_acc    


Philadelphia
entrance 0.14 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 2.8 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 5.45 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 7.27 mns
grid  300
grid  600
grid  1000
Philadelphia done 8.95 mns


Philadelphia
entrance_300        1 high        0.010640
                    2 medium      0.135305
                    3 low         0.221468
                    4 no          0.632587
entrance_600        1 high        0.131396
                    2 medium      0.316830
                    3 low         0.323869
                    4 no          0.227905
entrance_1000       1 high        0.492725
                    2 medium      0.305238
                    3 low         0.138870
                    4 no          0.063167
gravity**(1/2)_300  1 high        0.002397
                    2 medium      0.120023
                    3 low         0.174741
                    4 no          0.702838
gravity**(1/2)_600  1 high        0.037573
                    2 medium      0.289219
                    3 low         0.325834
                    4 no          0.347374
gravity**(1/2)_1000 1 high        0.234345
                    2 medium      0.419841
                    3 low         0.239585
                    4 no          0.106230
gravity**(1/3)_300  1 high        0.003063
                    2 medium      0.116408
                    3 low         0.178751
                    4 no          0.701778
gravity**(1/3)_600  1 high        0.042823
                    2 medium      0.287131
                    3 low         0.333182
                    4 no          0.336864
gravity**(1/3)_1000 1 high        0.255266
                    2 medium      0.416491
                    3 low         0.238278
                    4 no          0.089965
gravity**(1/5)_300  1 high        0.004124
                    2 medium      0.117990
                    3 low         0.189132
                    4 no          0.688754
gravity**(1/5)_600  1 high        0.056813
                    2 medium      0.296946
                    3 low         0.339163
                    4 no          0.307079
gravity**(1/5)_1000 1 high        0.319903
                    2 medium      0.393705
                    3 low         0.211442
                    4 no          0.074950

In [24]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'popreach'+score]] = str1.groupby('Park_No')[score,'pop' + score,'population'].sum()
            
            prk['walkha_person_' + score] = str1.groupby('Park_No')['walkha_person' + str2].mean()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
        
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_lines/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_lines/')
            
            prk.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_lines/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['geometry'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_geoms/'):
            os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_geoms/')
            
        prksc.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_geoms/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_csv/'):
            os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_csv/')
            
        prksc.to_csv('D:/Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[0][0])

park 300
park 600
park 1000
entrance 1.09 mns
park 300
park 600
park 1000
gravity**(1/2) 2.73 mns
park 300
park 600
park 1000
gravity**(1/3) 3.7 mns
park 300
park 600
park 1000
gravity**(1/5) 4.5 mns
Philadelphia done 4.5 mns


,tr_300,pop_tr_300,popreachtr_300,walkha_person_tr_300,tr_300_parks,tr_300route,tr_300Grid:Pentry,tr_300_access,tr_600,pop_tr_600,...,tr_600Grid:Pentry,tr_600_access,tr_1000,pop_tr_1000,popreachtr_1000,walkha_person_tr_1000,tr_1000_parks,tr_1000route,tr_1000Grid:Pentry,tr_1000_access
0,9768.904,105926.259,842.0,0.510050,76.0,"(LINESTRING (-75.239456 40.052769, -75.240065 ...","[1024, 1025, 1030, 1031, 1277, 1278, 1468, 172...",1 high,50087.526,5.747078e+05,...,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...",1 high,155153.171,2.026277e+06,5145.0,0.707083,354.0,"(LINESTRING (-75.2415597 40.0520621, -75.24156...","[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...",1 high
1,1316.272,32149.626,289.0,5.511059,12.0,"(LINESTRING (-75.2459535 39.9198435, -75.24618...","[23589, 23590, 23660, 23661, 23662, 23724, 237...",1 high,8916.996,2.044677e+05,...,"[23450, 23451, 23516, 23517, 23518, 23519, 235...",1 high,38785.030,1.039931e+06,3079.0,8.055026,100.0,"(LINESTRING (-75.2449838 39.924179, -75.245054...","[23147, 23148, 23228, 23229, 23230, 23306, 233...",1 high
2,45973.226,1867679.053,16187.0,1.898805,357.0,"(LINESTRING (-75.2586879 39.9851592, -75.25859...","[15244, 15245, 15246, 15370, 15371, 15372, 153...",1 high,222094.748,1.009671e+07,...,"[14822, 14823, 14824, 14972, 14973, 14974, 149...",1 high,631963.265,3.124307e+07,65777.0,2.462364,1233.0,"(LINESTRING (-75.2530653 39.9887453, -75.25336...","[14461, 14462, 14672, 14673, 14674, 14675, 146...",1 high
3,1613.461,107356.147,558.0,115.975062,10.0,"(LINESTRING (-75.151487 39.963505, -75.151726 ...","[18605, 18606, 18710, 18711, 18712, 18811, 188...",1 high,9003.715,4.180547e+05,...,"[18439, 18440, 18441, 18443, 18604, 18605, 186...",1 high,43659.881,1.851339e+06,5372.0,87.914943,127.0,"(LINESTRING (-75.1532837 39.9673479, -75.15352...","[17987, 17989, 17990, 17991, 17992, 18103, 181...",1 high
4,1526.436,12887.018,177.0,11.826702,14.0,"(LINESTRING (-75.1747348 40.0358392, -75.17395...","[5765, 5767, 5988, 6207, 6208, 6209, 6210, 642...",1 high,12351.783,2.184325e+05,...,"[5323, 5324, 5326, 5551, 5552, 5553, 5554, 576...",1 high,57917.494,1.372363e+06,4785.0,26.964493,166.0,"(LINESTRING (-75.17339 40.040145, -75.172542 4...","[4583, 4585, 4813, 4816, 4817, 4818, 4819, 482...",1 high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,1058.323,1.458021e+04,88.0,48.559103,5.0,"(LINESTRING (-75.251806 39.9526043, -75.251866...","[19901, 20004, 20110, 20214, 20318]:[7271, 727...",1 high
469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,42.505,8.501000e+01,4.0,0.738196,2.0,"(LINESTRING (-75.251806 39.9526043, -75.251866...","[19901, 20004]:[7283, 7283]",3 low
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [25]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['dissolve_key'], how = 'outer')
            g_csv = g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')]
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/')
                
            g_csv.to_csv('D:/Dumps/Scores output WPL-OSM/'+str(grid_block_size)
                         +'m grids/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y','geometry_m','Grid_m_centroid'])], geometry = 'geometry_x', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/')
                
            g_lines.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x','geometry_m','Grid_m_centroid'])], geometry = 'geometry_y', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/')
                
            g_geoms.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')
preference[0][0][0]

Philadelphia
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
all done


,level_0,index,geometry_x,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,...,tr_600,poptr_600,adj_tr_600,walk_area_ha600,walkha_person600,tr_1000,poptr_1000,adj_tr_1000,walk_area_ha1000,walkha_person1000
89440,57606.0,57606.0,"MULTILINESTRING ((-75.18732 40.05549, -75.1874...",1.0,1.099931e+08,1.098076e+08,normal way,169.102,2.0,50,...,345.438,13472.082,345.438,4.863697,8.018591,745.438,29072.082,447.2628,4.863697,8.018591
89676,48328.0,48328.0,"MULTILINESTRING ((-75.18696 40.05482, -75.1873...",1.0,1.099931e+08,1.098076e+08,normal way,249.619,3.0,51,...,302.210,12088.400,302.210,4.863697,8.224196,702.210,28088.400,421.3260,4.863697,8.224196
89966,48156.0,48156.0,"MULTILINESTRING ((-75.18605 40.05520, -75.1856...",1.0,5.542790e+09,5.542790e+09,normal way,247.760,5.0,52,...,308.680,29015.920,308.680,4.863697,19.326861,708.680,66615.920,425.2080,4.863697,19.326861
90222,35417.0,35417.0,"MULTILINESTRING ((-75.18506 40.05530, -75.1856...",1.0,1.098201e+08,5.542790e+09,normal way,246.156,5.0,53,...,303.794,20050.404,303.794,4.863697,13.569924,703.794,46450.404,422.2764,4.863697,13.569924
90553,35504.0,35504.0,"MULTILINESTRING ((-75.18446 40.05493, -75.1840...",1.0,1.103263e+08,1.097644e+08,normal way,227.707,2.0,54,...,332.101,11955.636,332.101,4.342151,8.290821,732.101,26355.636,439.2606,4.342151,8.290821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85481,1566642.0,1567879.0,"MULTILINESTRING ((-75.20587 40.05561, -75.2054...",1.0,1.098539e+08,5.284855e+09,normal way,242.938,2.0,34879,...,320.048,640.096,320.048,24.092847,0.083012,720.048,1440.096,432.0288,24.092847,0.083012
85486,32791.0,32791.0,"LINESTRING (-75.20540 40.05526, -75.20532 40.0...",1.0,8.038994e+09,5.284855e+09,normal way,186.579,1.0,34880,...,303.686,607.372,303.686,24.092847,0.083012,703.686,1407.372,422.2116,24.092847,0.083012
85488,1566694.0,1567931.0,None,1.0,3.021105e+09,3.021105e+09,normal way,0.000,0.0,34881,...,487.108,974.216,487.108,28.816132,0.069406,887.108,1774.216,532.2648,28.816132,0.069406
85490,1566695.0,1567932.0,None,1.0,3.021105e+09,3.021105e+09,normal way,0.000,0.0,34882,...,566.787,1133.574,566.787,28.816132,0.069406,966.787,1933.574,580.0722,28.816132,0.069406


In [26]:
print(round((time.time() - start) / 60,2),'mns')

174.81 mns
